In [ ]:
# package import
import chipwhisperer as cw
from sca_tools.scopes import *
from sca_tools.targets import SakuraX
from tqdm import tqdm

In [ ]:
# connect to the oscilloscope
VisaAddress = # your visa address here
scope = Oscilloscope(VisaAddress)
# setting the scope
# channel 3 is the trigger channel
scope.config_trigger_channel(TriggerMode.EDGE_FALL, 3, 1, 0)
# channel 1 is the trace channel, 2mV/div, trace period 4us
scope.config_trace_channel(1, milliVolt(2), -1, microSecond(4), delay = 0, impedance=50)


In [ ]:
# connect to the target
target = cw.target(scope, SakuraX)

In [ ]:
# create chipwhisperer project
project = cw.create_project("cpa_sakura", overwrite=True)

In [ ]:
# acquire traces
ktp = cw.ktp.Basic()
# generate a key
key = ktp.next_key()
print("key: ", key)

trace_count = 100

progress_bar = tqdm(total=trace_count, desc="Capturing traces")
while len(project.traces) < trace_count:
    text = ktp.next_text()
    trace = cw.capture_trace(scope, target, text, key)
    if trace is None:
        continue
    project.traces.append(trace)
    progress_bar.update(1)

In [ ]:
# save the project
project.save()